<a href="https://colab.research.google.com/github/xerxes786/Face_recognition-Using-SVM/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

In [ ]:
folder='/content/drive/MyDrive/a1_TrainingPhotos'
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   
    img = Image.open(os.path.join(folder,filename)) 

    img=img.resize([32,64]) 
    
    xyz, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, multichannel=True)
    
    X.append(xyz)
    
    if re.match("Amir*", filename):
      Yt=0
    elif re.match("Jaya*", filename):
      Yt=1
    elif re.match("Hir*", filename):
      Yt=2
    elif re.match("Anupam*", filename):
      Yt=3
    else:
      Yt=4

    Y.append(Yt)

In [ ]:
xyz.shape

(3780,)

In [ ]:
target_names=['AamirKhan', 'JayaBadhuri', 'HritikRoshan', 'AnupamKher']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
n_components = 25
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
import pickle as pk
pk.dump(pca, open("FR_1.pkl","wb"))

In [ ]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.1, probability=True)


In [ ]:
import joblib
joblib.dump(clf.best_estimator_, 'FR_svm_classifier.pkl', compress = 1)

['FR_svm_classifier.pkl']

In [ ]:
y_pred = clf.predict(X_test_pca)
y_pred

array([1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 0, 3, 0, 0, 2, 0, 2, 1, 0, 0,
       1, 0, 1, 0, 0, 3, 2, 2, 0, 1, 3, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 3, 1, 0, 0,
       2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 2,
       1, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 3, 1, 1, 0, 2, 2, 2, 1])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.94      0.84        49
           1       0.93      0.81      0.86        31
           2       0.69      0.53      0.60        17
           3       1.00      0.60      0.75        10

    accuracy                           0.80       107
   macro avg       0.84      0.72      0.76       107
weighted avg       0.82      0.80      0.80       107

